In [29]:
import random

import torch
from IPython import display
from d2l import torch as d2l
 #todo 弄懂它
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [30]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)
W.shape #784行 10列

torch.Size([784, 10])

In [31]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True #keepdim=True来保持指数和的列维度，这样就可以使用广播机制来将X_exp除以指数和，得到每行元素指数占比的矩阵，也就是每行的概率分布。
    return X_exp / partition  # 这里应用了广播机制

In [32]:
#这块就是一个测试，看看softmax函数好不好使
#最后真正使用的时候图片是784*1的输入，输出的X_prob是10个数，然后这十个数加起来等于1
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1) # 行总和为1

(tensor([[0.1863, 0.0484, 0.0493, 0.6434, 0.0725],
         [0.4348, 0.0214, 0.0349, 0.0514, 0.4575]]),
 tensor([1.0000, 1.0000]))

In [33]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b) #理解： 线性回归 + softmax激活

In [34]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

# 交叉熵以及对它举例解释


对于第一个样本，预测结果最大的是第三个类别（预测概率为 0.6），而标签是第三个类别，两者相同，因此交叉熵损失为 0。

对于第二个样本，预测结果最大的是第一个类别（预测概率为 0.3），而标签是第一个类别，两者相同，因此交叉熵损失为 0。

对于第三个样本，预测结果最大的是第一个类别（预测概率为 0.5），而标签是第二个类别，两者不同，因此交叉熵损失为 -log(0.1)。

对于第四个样本，预测结果最大的是第一个类别（预测概率为 0.8），而标签是第三个类别，两者不同，因此交叉熵损失为 -log(0.1)。

In [44]:
def cross_entropy(y_hat, y): #交叉熵损失函数 看看y_hat 和 y 之间差别有多大
    return - torch.log(y_hat[range(len(y_hat)), y])

y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5], [0.5, 0.1, 0.4], [0.8, 0.1, 0.1]])
y = torch.tensor([2, 0, 1, 2])
loss = cross_entropy(y_hat, y)
print(loss)

tensor([0.5108, 1.2040, 2.3026, 2.3026])


下面是一个用于计算模型准确率的函数。它采用两个参数 y_hat 和 y，分别是模型的预测值和实际标签值。
判断 y_hat 是否是二维数组（多分类问题），如果是则取最大值所在的下标作为预测的类别。
将 y_hat 和 y 相比较，得到一个布尔数组 cmp，其中每个元素表示相应位置上的预测是否正确。
统计 cmp 中为 True 的元素（即预测正确的数量）并将其转化为浮点数型，最后返回当前 batch 的准确率。
具体地，该函数会计算出模型在一个 batch 数据上的准确率，并将其返回。

In [36]:
def accuracy(y_hat, y): #输入的y_hat 和 y都是向量
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1) # Todo
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())


# accuracy函数细说使用指南
import numpy as np

# 构造样本数据
x = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
y = np.array([0, 1, 0])

# 假设这是一个二分类问题，且模型已经训练好并预测出了 y_hat。
y_hat = np.array([0, 1, 1])

# 计算准确率
acc = accuracy(y_hat, y)
print(acc) # 输出 0.67，即 2/3

y.numel() 是 PyTorch 中的一个函数，它返回张量 y 中元素的个数。
具体来说，如果 y 是一个 n1×n2×⋯×nk 的张量，则 y.numel() 的值为 n1×n2×⋯×nk。

这个函数在代码中被用来计算每个小批量数据的标签总数。在评估模型时，我们需要知道预测正确的样本数和总样本数以计算精度。
因此，在计算精度时，需要知道标签总数，而 y.numel() 可以方便地得到这个信息。

In [ ]:
def evaluate_accuracy(net, data_iter):  #@save
    '''
    :param net: 要评价的模型
    :param data_iter: 提供数据的迭代器
    :return:
    '''
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module): #判断net是不是torch.nn.Module类型的对象，如果是的话。进入评估模式
        net.eval()  # 将模型设置为评估模式，评估的时候，只需要进行前向计算，不用反向传播
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad(): # 在下面的with语境当中，torch会关闭梯度跟踪功能，然后出了with语句再开开
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1] # 正确预测数/预测总数 = 预测准确率

In [41]:
test = [0.0] * 10
test

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

# accuracy函数和evaluate_accuracy函数的主要区别
在于它们所用的数据集不同。accuracy函数只使用当前小批量数据，而evaluate_accuracy函数使用整个数据集。另外，evaluate_accuracy函数还需要判断模型是否处于评估模式，并对累加器进行初始化。

In [ ]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args): # 将数据累加
        self.data = [a + float(b) for a, b in zip(self.data, args)] # a代表data    b 代表args

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [38]:
evaluate_accuracy(net, test_iter)

0.1396